# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,berezovyy,51.6667,135.7000,-9.03,68,94,1.34,RU,1699720340
1,1,badger,64.8000,-147.5333,-8.07,73,100,1.54,US,1699720341
2,2,margaret river,-33.9500,115.0667,12.18,69,82,6.67,AU,1699720341
3,3,bengkulu,-3.8004,102.2655,23.97,93,14,1.59,ID,1699720341
4,4,muzaffarpur,26.1167,85.4000,23.07,53,0,2.39,IN,1699720341


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] > 15) & (
    city_data_df["Max Temp"] < 30) & (city_data_df["Cloudiness"] < 20) & (
    city_data_df["Humidity"] < 50), :]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna(how="any")

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,tilisarao,-32.7329,-65.2911,29.91,17,0,3.67,AR,1699720341
26,26,villa mercedes,-33.6757,-65.4578,28.70,21,0,5.35,AR,1699720293
126,126,arar,30.9753,41.0381,25.01,20,0,4.12,SA,1699720366
166,166,guerrero negro,27.9769,-114.0611,19.94,34,0,0.55,MX,1699720167
168,168,bayana,26.9000,77.2833,20.65,47,0,1.95,IN,1699720373
185,185,alice springs,-23.7000,133.8833,28.75,45,16,4.63,AU,1699720377
228,228,jampur,29.6441,70.5954,20.99,40,0,1.75,PK,1699720384
281,281,tataouine,32.9297,10.4518,19.84,40,1,1.66,TN,1699720395
316,316,berriane,32.8265,3.7669,25.47,36,1,4.01,DZ,1699720403
341,341,spring valley,36.1080,-115.2450,15.76,18,0,1.54,US,1699720409


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,tilisarao,AR,-32.7329,-65.2911,17,
26,villa mercedes,AR,-33.6757,-65.4578,21,
126,arar,SA,30.9753,41.0381,20,
166,guerrero negro,MX,27.9769,-114.0611,34,
168,bayana,IN,26.9000,77.2833,47,
185,alice springs,AU,-23.7000,133.8833,45,
228,jampur,PK,29.6441,70.5954,40,
281,tataouine,TN,32.9297,10.4518,40,
316,berriane,DZ,32.8265,3.7669,36,
341,spring valley,US,36.1080,-115.2450,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tilisarao - nearest hotel: No hotel found
villa mercedes - nearest hotel: La Valle
arar - nearest hotel: دار أوتيل Dar Otel
guerrero negro - nearest hotel: Plaza sal paraiso
bayana - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
jampur - nearest hotel: City Marriage Hall
tataouine - nearest hotel: No hotel found
berriane - nearest hotel: No hotel found
spring valley - nearest hotel: Club De Soleil
ghat - nearest hotel: فندق تاسيلي
copiapo - nearest hotel: Hotel Il Riposo
douz - nearest hotel: Louage station Douz
laborde - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
ataq - nearest hotel: الشارقة بلازا


,City,Country,Lat,Lng,Humidity,Hotel Name
5,tilisarao,AR,-32.7329,-65.2911,17,No hotel found
26,villa mercedes,AR,-33.6757,-65.4578,21,La Valle
126,arar,SA,30.9753,41.0381,20,دار أوتيل Dar Otel
166,guerrero negro,MX,27.9769,-114.0611,34,Plaza sal paraiso
168,bayana,IN,26.9000,77.2833,47,No hotel found
185,alice springs,AU,-23.7000,133.8833,45,Aurora Alice Springs
228,jampur,PK,29.6441,70.5954,40,City Marriage Hall
281,tataouine,TN,32.9297,10.4518,40,No hotel found
316,berriane,DZ,32.8265,3.7669,36,No hotel found
341,spring valley,US,36.1080,-115.2450,18,Club De Soleil


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)